# Matrix Factorization with Mllib in Spark

This notebook is a continuation of our previous Matrix Factorization notebook where we have built a MF engine with Keras. Here we use Matrix Factorization library from Mllib in Spark.

In [2]:
# !pip install pyspark

In [ ]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkContext

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
# set the random state
rs = 123

In [5]:
sc = SparkSession \
    .builder \
    .getOrCreate()

We load the same sampled dataframe used previously in our matrix factorization engine built with Keras

In [6]:
# load in the data

df = sc.read.csv("sample_df.csv.gz",header=True,inferSchema=True)

Let's have a look at our data

In [7]:
df.show(10)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
|     1|    253|   4.0|
|     1|    260|   4.0|
+------+-------+------+
only showing top 10 rows



We can easily print the schema and the number of rows/columns

In [8]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [9]:
print('number of rows : {}, number of columns :{}'.format(df.count(),len(df.columns)))

number of rows : 5399624, number of columns :3


In [10]:
# split into train and test
train, test = df.randomSplit([0.8, 0.2])

A matrix factorisation model trained by regularized alternating least-squares `ALS`.

`K` is the number of latent dimentionality and `epochs` is our number of iterations.

In [11]:
# train the model
K = 10
epochs = 10
model = ALS.train(train, K, epochs, nonnegative=True, seed=rs)

### Now we evaluate our model on the train and test sets

In [12]:
# train
x1 = train.rdd.map(lambda p: (p[0], p[1]))
x2 = model.predictAll(x1)
p = x2.map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = train.rdd.map(lambda r: ((r[0], r[1]), r[2])).join(p) # notice we had to map test such it has the same shape as p
# joins on first item: (user_id, movie_id)
# each row of result is: ((user_id, movie_id), (rating, prediction))
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("***** train rmse: %s *****" % mse**0.5)

***** train rmse: 0.6785713514938934 *****


In [13]:
# test
x1 = test.rdd.map(lambda p: (p[0], p[1]))
x2 = model.predictAll(x1)
p = x2.map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = test.rdd.map(lambda r: ((r[0], r[1]), r[2])).join(p)
# joins on first item: (user_id, movie_id)
# each row of result is: ((user_id, movie_id), (rating, prediction))
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("***** test rmse: %s *****" % mse**0.5)

***** test rmse: 0.8945413401724094 *****


We remember that the RMSE obtained for our Keras matrix factorization model on the test set was 0.8723, which is on pair with the current one.

----

below is just a simple check of how x1, x2, and ratesAndPreds look like.

In [14]:
print(x1)
x1.take(5)


PythonRDD[287] at RDD at PythonRDD.scala:53


[(2, 242), (2, 469), (2, 891), (3, 32), (3, 173)]

In [15]:
print(x2)
x2.take(5)

MapPartitionsRDD[278] at mapPartitions at PythonMLLibAPI.scala:1344


[Rating(user=65722, product=954, rating=2.4923833772943453),
 Rating(user=65722, product=948, rating=2.8576633229903474),
 Rating(user=65722, product=372, rating=1.3445525438204595),
 Rating(user=65722, product=858, rating=2.95247743531596),
 Rating(user=65722, product=110, rating=1.6773175328587708)]

In [16]:
print(p)
p.take(5)

PythonRDD[290] at RDD at PythonRDD.scala:53


[((65722, 954), 2.4923833772943453),
 ((65722, 948), 2.8576633229903474),
 ((65722, 372), 1.3445525438204595),
 ((65722, 858), 2.95247743531596),
 ((65722, 110), 1.6773175328587708)]

In [17]:
print(ratesAndPreds)
ratesAndPreds.take(5)

PythonRDD[292] at RDD at PythonRDD.scala:53


[((3, 512), (2.0, 3.455849970312213)),
 ((3, 593), (5.0, 4.321198629949171)),
 ((4, 356), (4.0, 3.582186548595037)),
 ((4, 594), (4.0, 2.5853824959762606)),
 ((5, 150), (5.0, 4.7181450491088555))]